In [1]:
# 400 meter coordinate accuracy
# 1923 earliest year
import pandas as pd
import geopandas as gpd
from dask import dataframe as dd
import dask_geopandas

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

In [2]:
# # dask visualizations
# from dask.distributed import Client

# client = Client()  # start distributed scheduler locally.
# client

/home/muskrat/miniconda3/envs/eco/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34887 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34887/status,
Dashboard: http://127.0.0.1:34887/status,Workers: 4
Total threads: 8,Total memory: 15.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38327,Workers: 4
Dashboard: http://127.0.0.1:34887/status,Total threads: 8
Started: Just now,Total memory: 15.36 GiB
Comm: tcp://127.0.0.1:46079,Total threads: 2
Dashboard: http://127.0.0.1:40653/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:38881,


In [ ]:
# read in cleaned parquet
df_path = input("Enter the file path: ")

df = dd.read_parquet(df_path)

In [ ]:
# read in eco_map.geojson
map_path = input("Enter the file path: ")

map = gpd.read_file(map_path)

In [ ]:
df

In [ ]:
map.head()

In [ ]:
map.crs

In [ ]:
# convert to dask_geopandas df
df_gpd = dask_geopandas.from_dask_dataframe(df)

In [ ]:
del df, df_path


In [ ]:
df_gpd = df_gpd.set_geometry(
    dask_geopandas.points_from_xy(df_gpd, "decimalLongitude", "decimalLatitude")
)

In [ ]:
df_gpd = df_gpd.drop(["decimalLatitude", "decimalLongitude"], axis=1)

In [ ]:
df_gpd = df_gpd.set_crs("EPSG:4326")

# if get stuck might be useful at some point
# df_gpd.drop(df_gpd[df_gpd["geometry"].is_empty].index, inplace=True)

# df_gpd["geometry"] = df_gpd["geometry"].apply(wkt.loads)
# df_gpd = gpd.GeoDataFrame(df_gpd, crs="EPSG:4326")


In [ ]:
df_gpd.crs

In [ ]:
df_gpd = df_gpd.compute()

# df_gpd.head()

In [ ]:
df_gpd = df_gpd.sjoin(map, predicate="within")

In [ ]:
df_gpd

In [ ]:
del map, map_path


In [ ]:
df_gpd = df_gpd.drop(["index_right", "geometry", "name", "TYPE"], axis=1)


In [ ]:
df_gpd

In [ ]:
# combine license and rightsHolder into dict and merge in single column
rights = ["license", "rightsHolder"]
df_gpd["rights"] = df_gpd[rights].to_dict(orient="records")
df_gpd = df_gpd.drop(["license", "rightsHolder"], axis=1)

In [ ]:
df_gpd

In [ ]:
group_id = df_gpd.groupby("species")["unique_id"].apply(list).reset_index()


In [ ]:
group_rights = df_gpd.groupby("species")["rights"].apply(list).reset_index()


In [ ]:
# ecos5 = df_gpd.groupby(['scientific_name'], as_index=False)


# %%


# ecos5 = ecos5.apply(lambda x: x)
unique_species = df_gpd.drop_duplicates(subset=["species"])
unique_species = unique_species.drop(["unique_id", "rights"], axis=1)


# %%


In [ ]:
del df_gpd, rights


In [ ]:
merged_groups = pd.merge(group_id, group_rights, on="species", how="left").reindex(
    columns=["species", "unique_id", "rights"]
)


In [ ]:
del group_id, group_rights


In [ ]:
total_merge = pd.merge(merged_groups, unique_species, on="species", how="left")


In [ ]:
del merged_groups, unique_species


In [ ]:
# remove duplicates values from list in unique_id column
total_merge["unique_id"] = total_merge["unique_id"].apply(set).apply(list)


In [ ]:
total_merge

In [ ]:
rights = total_merge["rights"].to_list()


In [ ]:
# tester of removing duplicate dicts from list
# rights_43 = rights[43]

# only keep unique values from rights_43 list
# rights_43 = [i for n, i in enumerate(rights_43) if i not in rights_43[n + 1:]]


In [ ]:
# VERY SLOW!!! TOOK 4 HOURS ON 8500 ROWS(10 MILLION COMPRESSED)
# remove duplicate dictionaries from list of lists
for j in range(len(rights)):
    print(j)
    rights[j] = [i for n, i in enumerate(rights[j]) if i not in rights[j][n + 1 :]]

In [ ]:
# check some rights values. doesn't show up properly in variable explorer
rights[103]

In [ ]:
# convert to series
rights = pd.Series(rights, name="rights")

total_merge["rights"] = rights

In [ ]:
del rights, j


In [ ]:
# rename species column to scientific_name
total_merge = total_merge.rename(columns={"species": "scientific_name"})

In [ ]:
total_merge = total_merge[
    [
        "kingdom",
        "phylum",
        "class",
        "order",
        "family",
        "genus",
        "scientific_name",
        "unique_id",
        "rights",
    ]
]


In [ ]:
total_merge

In [ ]:
# read cleaned df into a parquet file where user can input the file path
df_spatial_path = input("Enter the file path: ")


# write df to parquet file using pandas to_parquet
total_merge.to_parquet(df_spatial_path)